In [1]:
#imports
import optuna
from model import *

In [4]:
## Optuna Function
def objective(trial):

    # Suggest hyperparameters
    # Restrict d_model and nhead to best from first optuna to get better insights (no inf)
    d_model = 64
    nhead = 4
    
    seasons = trial.suggest_int("seasons", 0, 1)
    days = trial.suggest_int("weekdays", 0, 1)
    weekday_weekend = trial.suggest_int("weekday_weekend", 0, 1)
    months = trial.suggest_int("months", 0, 1)
    min_demand = trial.suggest_int("min_demand", 0, 1)
    avg_demand = trial.suggest_int("avg_demand", 0, 1)
    max_demand = trial.suggest_int("max_demand", 0, 1)
    avg_temp = trial.suggest_int("avg_temp", 0, 1)
    max_temp = trial.suggest_int("max_temp", 0, 1)
    min_temp = trial.suggest_int("min_temp", 0, 1)
    hd = trial.suggest_int("hd", 0, 1)
    cd = trial.suggest_int("cd", 0, 1)
    features = ['sum_30_min_demand']  # Always include sum_30_min_demand
    cases = [seasons, days, weekday_weekend, months, min_demand, avg_demand, max_demand, avg_temp, max_temp, min_temp, hd, cd]
    feature_names = [['is_summer', 'is_autumn', 'is_winter', 'is_spring'], 
                     ['is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday'], 
                     ['is_weekday', 'is_weekend'], 
                     ['is_jan', 'is_feb', 'is_mar', 'is_apr', 'is_may', 'is_jun', 'is_jul', 'is_aug', 'is_sep', 'is_oct', 'is_nov', 'is_dec'], 
                     ['min_30_min_demand'],
                     ['avg_30_min_demand'],
                     ['max_30_min_demand'], 
                     ['avg_temp'], 
                     ['max_temp'], 
                     ['min_temp'], 
                     ['hd_next_24h'], 
                     ['cd_next_24h']]
    for i in range(len(cases)):
        if cases[i] == 1:
            features += feature_names[i]

    params = {
        'learning_rate': 0.00306547286804383,
        'batch_size': 14,
        'seq_length': 18,
        'transformer_encoder_layer_params': {
            'd_model': 64,
            'nhead': 4,
            'dim_feedforward': 192,
            'dropout': 0.07625744142022907,
            'activation': 'relu'
        },
        'transformer_layer_params': {
            'num_layers': 1,
        },
        'dataset': '2016-2019',
        'train_test_split': 'prior:2019',
        'features': features,
        'visualise': False
    }
    
    return median_model(params, 5)['results']['test_mape']

In [5]:
study = optuna.create_study(storage="sqlite:///db.sqlite3", study_name="feature_selection, 2016-2019", load_if_exists=True)
study.optimize(objective, n_trials=100, n_jobs=1)

study.best_params

[I 2025-10-02 12:57:15,885] Using an existing study with name 'feature_selection, 2016-2019' instead of creating a new one.
[I 2025-10-02 12:58:31,909] Trial 1 finished with value: 3.677036207596364 and parameters: {'seasons': 0, 'weekdays': 0, 'weekday_weekend': 1, 'months': 1, 'min_demand': 1, 'avg_demand': 1, 'max_demand': 1, 'avg_temp': 0, 'max_temp': 0, 'min_temp': 0, 'hd': 1, 'cd': 0}. Best is trial 1 with value: 3.677036207596364.
[I 2025-10-02 13:00:02,103] Trial 2 finished with value: 3.8483670460306123 and parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 0, 'months': 0, 'min_demand': 1, 'avg_demand': 1, 'max_demand': 1, 'avg_temp': 1, 'max_temp': 0, 'min_temp': 1, 'hd': 1, 'cd': 0}. Best is trial 1 with value: 3.677036207596364.
[I 2025-10-02 13:01:20,308] Trial 3 finished with value: 3.696485777778871 and parameters: {'seasons': 1, 'weekdays': 0, 'weekday_weekend': 0, 'months': 1, 'min_demand': 0, 'avg_demand': 0, 'max_demand': 1, 'avg_temp': 0, 'max_temp': 1, 'm

{'seasons': 1,
 'weekdays': 1,
 'weekday_weekend': 1,
 'months': 0,
 'min_demand': 0,
 'avg_demand': 1,
 'max_demand': 0,
 'avg_temp': 1,
 'max_temp': 1,
 'min_temp': 0,
 'hd': 0,
 'cd': 1}

In [1]:
study.best_params

NameError: name 'study' is not defined